In [0]:
import torch

In [0]:
from sklearn.datasets import load_iris

iris = load_iris()
x_data=iris.data
y_data=iris.target

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.33, random_state=42)

In [0]:
# Convert to pytorch dataset
tensor_x_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)
tensor_x_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

from torch.utils import data
train_dataset = data.TensorDataset(tensor_x_train, tensor_y_train)
test_dataset = data.TensorDataset(tensor_x_test, tensor_y_test)

In [0]:
# Dataset Loader (Input Batcher)
batch_size = 10
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
import torch.nn as nn
from collections import OrderedDict

class IrisNN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(IrisNN, self).__init__()

    self.input_size = input_size
    self.num_classes = num_classes

    self.net = nn.Sequential(OrderedDict([
            ('linear_1', nn.Linear(input_size, 75)),
            ('Relu1', nn.ReLU()),

            ('linear_2', nn.Linear(75, 50)),
            ('Relu2', nn.ReLU()),
            
            ('linear_3', nn.Linear(50, num_classes)),
        ]))

  def forward(self, x):
    out = self.net(x)
    return out

In [0]:
num_classes = 3
input_size = 4

model = IrisNN(input_size, num_classes)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10

# Copy all model parameters to the GPU
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
  total_loss = 0.0

  for (inputs, labels) in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)

    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print("Epoch %d, Loss=%.4f" % (epoch+1, total_loss/len(train_loader)))

  


Epoch 1, Loss=1.1027
Epoch 2, Loss=0.9884
Epoch 3, Loss=0.9140
Epoch 4, Loss=0.8349
Epoch 5, Loss=0.7433
Epoch 6, Loss=0.6449
Epoch 7, Loss=0.5671
Epoch 8, Loss=0.4970
Epoch 9, Loss=0.4435
Epoch 10, Loss=0.4015


In [0]:
def accuracy(model, data_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in data_loader:
            inputs = inputs.to(device)     
            
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            correct += (predicted.cpu() == labels).sum().item()
            total += labels.size(0)
            
    acc = correct / total
    return acc

In [0]:
accuracy(model, train_loader, device)

0.94

In [0]:
accuracy(model, test_loader, device)

1.0

In [0]:
# torch.save(model, "my_model.pt")
# my_model_loaded = torch.load("my_model.pt")